# 图像特征计算

## 一、色彩丰富度
色彩丰富度标准：Hasler and Süsstrunk 的研究将颜色丰富度划分为七级：
> 1.无      (Not colorful)  
> 2.稍微    (Slight colorful)  
> 3.适度    (Moderately colorful)  
> 4.平均    (Averagely colorful)  
> 5.非常    (Quite colorful)  
> 6.高度    (Highly colorful)  
> 7.极端    (Extremely colorful)  

In [420]:
import cv2
import numpy as np

In [421]:
def image_colorfulness(image):
    # 将图片划分为R，G，B三色通道（这里得到的R，G，B喂向量而非标量）
    (B,G,R) = cv2.split(image.astype("float"))

    # rg = R - G
    rg = np.absolute(R - G)

    # yb = 0.5 * (R + G) - B
    yb = np.absolute(0.5 * (R + G) - B)

    # 计算rg和yb的平均值和标准差
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))

    # 计算rgyb的标准差和平均值
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))

    # 返回颜色丰富度C（范围 0-100）
    return stdRoot + (0.3 * meanRoot)

**原图**  
![Origin](./picTest/Origin.jpg)

**原图，黑白50，黑白99**  
![Black](./picTest/Black1.jpg)

In [422]:
image = cv2.imread('./picTest/Origin.jpg')
print(image_colorfulness(image))

28.673850693563416


<!-- **黑白50**
![Black50](./picTest/Black50.jpg) -->

In [423]:
imageBlack50 = cv2.imread('picTest/Black50.jpg')
print(image_colorfulness(imageBlack50))

14.404272186323636


<!-- **黑白99**
![Black99](./picTest/Black99.jpg) -->

In [424]:
imageBlack99 = cv2.imread('picTest/Black99.jpg')
print(image_colorfulness(imageBlack99))

0.2821375870571007


## 二、图像对比度

### 2.1定义
对比度：通俗地讲就是亮暗的拉伸对比程度，通常表现了图像画质的清晰程度，对比度的计算公式为：  
![20220421160459](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220421160459.png)  
**计算方法**：  
按照四近邻计算，如第一个小括号：以第一行第一列为中心，上下左右分别与这个中心值相减再平方，然后加在一起，即：(2-1)^2+(3-1)^2；第二个括号即：(1-3)^2+(9-3)^2+(1-3)^2。最后除以的48为总的平方次个数。  
![20220421163003](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220421163003.png)

In [425]:
from cv2 import cv2
import numpy as np

In [426]:
def contrast(img0):
    # 将图片转化为灰度图片
    img1 = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)
    m,n = img1.shape
    # 图片矩阵向外扩散一个像素方便计算,除以1.0的目的是转化为float类型，便于后续计算
    img1_ext = cv2.copyMakeBorder(img1,1,1,1,1,cv2.BORDER_REPLICATE) / 1.0
    rows_ext,cols_ext = img1_ext.shape
    b = 0.0
    for i in range(1,rows_ext-1):
        for j in range(1,cols_ext-1):
            b += ((img1_ext[i,j]-img1_ext[i,j+1])**2 + (img1_ext[i,j]-img1_ext[i,j-1])**2 + (img1_ext[i,j]-img1_ext[i+1,j])**2 + (img1_ext[i,j]-img1_ext[i-1,j])**2)
    
    cg = b/(4*(m-2)*(n-2)+3*(2*(m-2)+2*(n-2))+2*4)
    return cg

In [427]:
cg = contrast(image)
cg

251.24037533553346

**对比度50，100，-50，-100**  
![Compare](./picTest/Compare.jpg)

In [428]:
imageCompare50 = cv2.imread('picTest/Compare50.jpg')
cgCompare50 = contrast(imageCompare50)
cgCompare50

337.0040768985861

In [429]:
imageCompare100 = cv2.imread('picTest/Compare100.jpg')
cgCompare100 = contrast(imageCompare100)
cgCompare100

430.84536589055233

In [430]:
imageComparef50 = cv2.imread('picTest/Compare-50.jpg')
cgComparef50 = contrast(imageComparef50)
cgComparef50

197.5235889026114

In [431]:
imageComparef100 = cv2.imread('picTest/Compare-100.jpg')
cgComparef100 = contrast(imageComparef100)
cgComparef100

154.68938670347742

## 三、色度（色调），饱和度与亮度

**HSV颜色模型**  
![20220421231709](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220421231709.png)

**①色调H（Hue）**  
用角度度量，取值范围为0°~360°，从红色开始按逆时针方向计算，红色为0°，绿色为120°，蓝色为240°；他们的补色为黄色60°，情色180°，品红300°，0°- 359°时颜色会依次变换。  
![20220422002940](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220422002940.png)

**②饱和度S（Saturation）**  
饱和度S表示颜色接近光谱色的程度，一种颜色可以看成是某种光谱色与白色混合的结果。其中光谱色所占的比例愈大，颜色接近光谱色的程度就愈高，颜色的饱和度也就愈高。饱和度高，颜色则深而艳。光谱色的白光成分为0，饱和度达到最高。通常取值范围为0%～100%，值越大，颜色越饱和。  
![20220422003154](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220422003154.png)

**③亮度V（Value）**  
图像的亮度通俗的理解便是图像的明暗程度，数字图像f(x,y)=i(x,y)r(x,y)，如果灰度值在[0,255]之间，则f值越接近0亮度越低，f值越接近255亮度越高。  
![20220422003241](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220422003241.png)  
![20220421230721](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220421230721.png)  
上图中的白色和红色都是右边相对于左边增加了亮度，可以看出图像右边相对于左边亮度有了一个整体的提升，这里只是对亮度做了一个小弧度的提升，接着尝试着将亮度提升的更高，如下图：  
![20220421230839](https://raw.githubusercontent.com/ZZh2333/picgoResource/main/img/20220421230839.png)  
需要注意的是，在改变亮度的同时也影响了图片的饱和度、对比度。原因是过度增加亮度导致阴影赶上了高光，因为最大灰度值是固定的，所以最低灰度值快赶上了最大灰度值，因此影响了图片的饱和度、对比度。

In [432]:
import cv2
import os.path
import numpy as np
import pandas as pd

In [433]:
imageHSV = cv2.imread('picTest/Origin.jpg')
hsv = cv2.cvtColor(imageHSV ,cv2.COLOR_RGB2HSV)
H,S,V = cv2.split(hsv)
# 取色度非0的值
h = H.ravel()[np.flatnonzero(H)]
# 计算平均色度
average_h = sum(h)/len(h)
print("色度为："+str(average_h))
# 取亮度非0的值
v = V.ravel()[np.flatnonzero(V)]
# 计算平均亮度
average_v = sum(v)/len(v)
print("饱和度为："+str(average_v))
# 取饱和度非0的值
s = S.ravel()[np.flatnonzero(S)]
# 计算平均饱和度
average_s = sum(s)/len(s)
print("亮度为："+str(average_s))

色度为：82.42670616804097
饱和度为：144.62686349734884
亮度为：63.94017816833792
